# Predictions


In [1]:
import os
import ast

import base

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.externals import joblib

plt.rcParams['font.size'] = 16
plt.rcParams['axes.facecolor'] = 'white'

%matplotlib inline

In [2]:
def fetch_models(path_to_models, labels):
    """Read model dumps from disk."""
    
    models = {}
    for num, path in enumerate(path_to_models):
        models[labels[num]] = joblib.load(path) 
    
    return models

In [3]:
# Paths to ref cluster indications.
path_target_genes = './../data/test/emQTL_Cluster_genes.txt'
path_target_cpgs = './../data/test/emQTL_Clusters_CpGs.txt'

In [4]:
# Indicator labels for classes of reference data.
ref_labels = [
    'orig_pvalues', 'sel_pvalues','orig_pcc', 'sel_pcc'
]

In [5]:
# Read experimental data
ref_data = {
    # NOTE: Transpose to (Cpgs x genes).
    ref_labels[0]: pd.read_csv(
        './../data/train/orig_pvalues_prep.csv', sep=',', index_col=0
    ).T,
    ref_labels[1]: pd.read_csv(
        './../data/train/sel_pvalues_prep.csv', sep=',', index_col=0
    ),
    # NOTE: Transpose to (Cpgs x genes).
    ref_labels[2]: pd.read_csv(
        './../data/train/orig_pcc_prep.csv', sep=',', index_col=0
    ).T,
    ref_labels[3]: pd.read_csv(
        './../data/train/sel_pcc_prep.csv', sep=',', index_col=0
    ),
}

## Models: sklearn

In [6]:
path_to_skmodels = [
    './../model_dumps/sk_orig_prep_pvalues.pkl',
    './../model_dumps/sk_sel_prep_pvalues.pkl',
    './../model_dumps/sk_orig_prep_pcc.pkl',
    './../model_dumps/sk_sel_prep_pcc.pkl',
]
sk_models = fetch_models(path_to_skmodels, ref_labels)

In [7]:
# Fit selected models to data.
for name, model in sk_models.items():
    model.fit(ref_data[name])

In [43]:
class Biclusters:
    """Representation of a set of predicted
    biclusters."""

    def __init__(self, rows, cols, data):

        self.rows = rows
        self.cols = cols
        self.data = data
        
        # NOTE: Sets attributes.
        self._setup()
        
    @property
    def nbiclusters(self):
        
        return self._nbiclusters
    
    @nbiclusters.setter
    def nbiclusters(self, value):

        if np.shape(self.rows)[0] == np.shape(self.cols)[0]:
            self._nbiclusters = value
        else:
            raise RuntimeError('Sample clusters: {}, ref clusters {}'
                               ''.format(sample, ref))

    def _setup(self):
        
        self.nrows, self.ncols = np.shape(self.data)
        self.nbiclusters = np.shape(self.rows)[0]

        return self

    @property
    def indicators(self):
        """Determine coordiantes of row and column indicators
        for each bicluster.
        """

        row_idx, col_idx = [], []
        for cluster_num in range(self.nbiclusters):

            rows_bools = self.rows[cluster_num, :] != 0
            cols_bools = self.cols[cluster_num, :] != 0

            rows = [index for index, elt in enumerate(rows_bools) if elt]
            cols = [index for index, elt in enumerate(cols_bools) if elt]

            row_idx.append(rows), col_idx.append(cols)

        return row_idx, col_idx

    @property
    def stats(self):
        """Compute max, min and std from data points 
        included in biclusters.
        """
        
        row_idx, col_idx = self.indicators 
        data_size = np.size(self.data)
        
        stats = {}
        for num in range(self.nbiclusters):
            
            _row_cluster = self.data.values[row_idx[num], :]
            cluster = _row_cluster[:, col_idx[num]] 
            if np.any(cluster):
                stats[num] = {
                    'max': np.max(cluster),
                    'min': np.min(cluster),
                    'std': np.std(cluster),
                    'rel_size': np.size(cluster) / data_size,
                    'zeros': int(np.count_nonzero(cluster==0))
                }
            else:
                pass
            
        df_stats = pd.DataFrame(stats).T
        df_stats.index.name = 'num'

        return df_stats
    
    @property
    def labels(self):
        """Assign row and column labels to biclusters."""
        
        genes = np.array(self.data.columns, dtype=object)
        cpgs =  np.array(self.data.index, dtype=object)
        
        row_idx, col_idx = self.indicators
        
        row_labels, col_labels = [], []
        for num in range(self.nbiclusters):
            row_labels.append(cpgs[row_idx[num]])
            col_labels.append(genes[col_idx[num]])
            
        return row_labels, col_labels
    
    def to_disk(self):
        
        pass

In [49]:
# Create Bicluster instances tracking detected clusters 
sk_biclusters = {}
for label in ref_labels:
    rows = sk_models[label].rows_
    cols = sk_models[label].columns_

    assert np.shape(rows)[0] == np.shape(cols)[0]
    
    sk_biclusters[label] = Biclusters(
        rows=rows, cols=cols, data=ref_data[label]
    )

### Bicluster statistics

In [31]:
for label in ref_labels:
    print('{0}\n{1}'.format(label, '-' * 40))
    print(sk_biclusters[label].stats)
    print()

orig_pvalues
----------------------------------------
           max           min  rel_size       std  zeros
num                                                    
0    57.268668  9.188604e-06  0.102633  7.193694    0.0
1    73.435290  9.248932e-07  0.367334  5.768430    0.0
2    41.719159  4.404799e-04  0.003666  6.030900    0.0

sel_pvalues
----------------------------------------
           max  min  rel_size       std       zeros
num                                                
0    73.435290  0.0  0.940773  2.986360  68334504.0
1    63.915840  0.0  0.000110  3.572772      7989.0
2    43.548003  0.0  0.000084  3.782657      6106.0

orig_pcc
----------------------------------------
          max       min  rel_size       std  zeros
num                                               
0    0.822428 -0.736304  0.178013  0.143568    0.0
1    0.852443 -0.738278  0.085744  0.277983    0.0
2    0.791034 -0.685308  0.080356  0.195047    0.0

sel_pcc
-------------------------------------

### Reference comparison

In [32]:
import ast
import numpy as np


class References:
    """Representation of a set of reference biclsuters."""

    @classmethod
    def from_files(cls, path_to_cpgs, path_to_genes, num_ref_clusters=2):
        
        # Read target CpG data.
        target_cpgs = {str(num + 1): [] for num in range(num_ref_clusters)}
        with open(path_to_cpgs, 'r') as cpgfile:

            cpg_contents = cpgfile.read().split('\n')
            # Skip header line.
            for row in cpg_contents[1:]:
                try:
                    value, idx, _ = row.split()
                    target_cpgs[idx].append(ast.literal_eval(value))
                except:
                    pass
                    
        # Read target gene data.
        target_genes = {str(num + 1): [] for num in range(num_ref_clusters)}
        with open(path_to_genes, 'r') as genefile:

            gene_contents = genefile.read().split('\n')
            # Skip header line.
            for row in gene_contents[1:]:
                try:
                    value, idx = row.split()
                    target_genes[idx].append(ast.literal_eval(value))
                except:
                    pass

        return References(cpgs=target_cpgs, genes=target_genes)

    def __init__(self, cpgs, genes):

        self.cpgs = cpgs
        self.genes = genes

    @property
    def cpgs1(self):

        return self.cpgs['1']

    @property
    def cpgs2(self):

        return self.cpgs['2']

    @property
    def genes1(self):

        return self.genes['1']

    @property
    def genes2(self):

        return self.genes['2']

In [205]:
def recovery_score(true, pred):
    """The percentage of true items among the predicted
    items."""

    try:
        frac = np.isin(pred, true).sum() / np.size(true)
        result = np.round(frac * 100, decimals=1)
    except:
        result = np.nan
    
    return result

    
def relevance_score(true, pred):
    """The percentage the true predicted items
    constitute in the total predicted population."""
    prin
    try:
        frac = np.isin(true, pred).sum() / np.size(pred)
        result = np.round(frac * 100, decimals=1)
    except:
        result = np.nan
    print(frac, result)
    return result
    
    
def compare_clusters(pred_cluster, target_cluster):
    
    cpgs, genes = pred_cluster.labels        
    
    scores = np.zeros((len(cpgs) * 2, 4), dtype=float)

    prev = 0
    for num, _cpgs in enumerate(cpgs):
        
        cpg_id, gene_id = (num + prev), (num + prev + 1)
        
        scores[cpg_id, 0] = recovery_score(target_cluster.cpgs1, _cpgs)
        scores[cpg_id, 1] = relevance_score(target_cluster.cpgs1, _cpgs)
        scores[cpg_id, 2] = recovery_score(target_cluster.cpgs2, _cpgs)
        scores[cpg_id, 3] = relevance_score(target_cluster.cpgs2, _cpgs)
        
        scores[gene_id, 0] = recovery_score(target_cluster.genes1, genes[num])
        scores[gene_id, 1] = relevance_score(target_cluster.genes1, genes[num])
        scores[gene_id, 2] = recovery_score(target_cluster.genes2, genes[num])
        scores[gene_id, 3] = relevance_score(target_cluster.genes2, genes[num])

        prev += 1
        
    idx = pd.MultiIndex.from_product(
        [('ref1', 'ref2'), ('recovery', 'relevance')], names=('cluster', 'score')
    )
    cols = pd.MultiIndex.from_product(
        [np.arange(num + 1) + 1, ('cpgs', 'genes')], names=('num', 'kind')
    )
    df = pd.DataFrame(scores.T, index=idx, columns=cols)
    
    return df

In [206]:
# Reference clusters.
#ref_cls = References.from_files(path_target_cpgs, path_target_genes)

#biclust_cpgs = sk_biclusters[ref_labels[0]].labels[0]
#biclust_genes = sk_biclusters[ref_labels[0]].labels[1]
df = compare_clusters(sk_biclusters[ref_labels[-1]], ref_cls)
df

0.967021276596 96.7
0.000531914893617 0.1
nan nan
nan nan
0.0658951837822 6.6
0.089164550639 8.9
0.0618238021638 6.2
0.0795981452859 8.0
0.0 0.0


/Users/severinlangberg/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in long_scalars


0.879372738239 87.9
0.0131578947368 1.3
0.855263157895 85.5


num                   1           2           3      
kind               cpgs genes  cpgs genes  cpgs genes
cluster score                                        
ref1    recovery   53.5   0.0  46.5  99.4   0.0   0.6
        relevance  96.7   NaN   6.6   6.2   0.0   1.3
ref2    recovery    0.0   0.0  59.5  76.0  40.5  24.0
        relevance   0.1   NaN   8.9   8.0  87.9  85.5

In [196]:
pred_cluster = sk_biclusters[ref_labels[0]]
cpgs, genes = pred_cluster.labels

# Reference clusters.
ref_clusters = References.from_files(path_target_cpgs, path_target_genes)

In [197]:
for label in ref_labels:
    print('{0}\n{1}'.format(label, '-' * 40))
    print(compare_clusters(sk_biclusters[label], ref_clusters))
    print()

orig_pvalues
----------------------------------------
num                    1            2          3      
kind                cpgs  genes  cpgs genes cpgs genes
cluster score                                         
ref1    recovery   100.0  100.0   0.0   0.0  0.0   0.0
        relevance   31.4   23.1   0.0   0.0  0.0   0.0
ref2    recovery     1.8    7.7  98.2  92.3  0.0   0.0
        relevance    0.6    3.0  22.6  14.5  0.0   0.0

sel_pvalues
----------------------------------------
num                    1           2          3      
kind                cpgs  genes cpgs genes cpgs genes
cluster score                                        
ref1    recovery   100.0  100.0  0.0   0.0  0.0   0.0
        relevance   12.4    6.4  0.0   0.0  0.0   0.0
ref2    recovery   100.0  100.0  0.0   0.0  0.0   0.0
        relevance   13.2   10.7  0.0   0.0  0.0   0.0

orig_pcc
----------------------------------------
num                   1           2           3      
kind               cpgs 

/Users/severinlangberg/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in long_scalars


num                   1           2           3      
kind               cpgs genes  cpgs genes  cpgs genes
cluster score                                        
ref1    recovery   53.5   0.0  46.5  99.4   0.0   0.6
        relevance  96.7   NaN   6.6   6.2   0.0   1.3
ref2    recovery    0.0   0.0  59.5  76.0  40.5  24.0
        relevance   0.1   NaN   8.9   8.0  87.9  85.5



### Visualization

## Models: R

In [134]:
# Collect pickled wrapped R models
path_to_rmodels = [
    './../model_dumps/r_orig_prep_pcc.pkl',
    './../model_dumps/r_sel_prep_pvalues.pkl',
    './../model_dumps/r_orig_prep_pcc.pkl',
    './../model_dumps/r_sel_prep_pcc.pkl',
]
r_models = fetch_models(path_to_rmodels, ref_labels)

In [135]:
for name, model in r_models.items():
    # NOTE: Convert <pandas.DataFrame> to <numpy.ndarray>
    model.fit(ref_data[name].values)

In [136]:
# Create Bicluster instances tracking detected clusters 
r_biclusters = {}
for label in ref_labels:
    rows = r_models[label].rows_
    cols = r_models[label].columns_

    assert np.shape(rows)[0] == np.shape(cols)[0]
    
    r_biclusters[label] = Biclusters(
        rows=rows, cols=cols, data=ref_data[label]
    )

### Bicluster statistics

In [145]:
for label in ref_labels:
    print('{0}\n{1}'.format(label, '-' * 40))
    print(r_biclusters[label].stats)
    print()

orig_pvalues
----------------------------------------
          max       min  rel_size       std  zeros
num                                               
0    2.886458  0.000004  0.001305  0.330803    0.0
1    2.651763  0.000014  0.000695  0.334991    0.0

sel_pvalues
----------------------------------------
     max  min  rel_size  std       zeros
num                                     
0    0.0  0.0  0.471879  0.0  34646520.0
1    0.0  0.0  0.067740  0.0   4973617.0

orig_pcc
----------------------------------------
          max       min  rel_size      std  zeros
num                                              
0    0.852443 -0.866965       1.0  0.32326    0.0

sel_pcc
----------------------------------------
          max       min  rel_size      std       zeros
num                                                   
0    0.852443 -0.866965       1.0  0.06393  72682896.0



In [ ]:
# NOTE: Can be wrapped into loop for multiple predictions.
def predict(model, data):
    """Predict biclusters from a dataset."""
        
    # Fit model to data to determine biclusters.        
    model.fit(data)
    
    # Reconstruct data matrices by sorting data according to
    # predicted biclusters.
    sorted_col_ids = np.argsort(model.column_labels_)
    row_sort_array = data[np.argsort(model.row_labels_)]
    return row_sort_array[:, sorted_col_ids]

In [ ]:
def gen_graphics(data, title, out_path):
    """Generate a heatmap and save figure to disk."""
    
    plt.figure(figsize=(10, 10))
    plt.title(title)
    sns.heatmap(
        data, robust=True, 
        cmap=plt.cm.RdBu_r, fmt='f', 
        vmin=np.min(data), vmax=np.max(data),
    )
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(out_path)
    
    return None

In [ ]:
# NOTE: Pass trained models with rows_ and cols_ attribtues.
def collect_cluster_members(models, references, ref_data):
    """Determine biclsuter row and column indicators, and collect
    bicluster gene and CpG members."""
    
    # Collect bicluster indicators for each detected bicluster.
    biclusters = {}
    for name, model in models.items():
        biclusters[name] = cluster_indices(
            model.rows_, model.columns_
        )
    # Convert cluster indices to cpG and gene labels.
    cluster_members = {}
    # For each class of reference data
    for data_class in references:
        # For each bicluster detected in reference data
        cluster_members[data_class] = {}
        for cluster_num, bicluster in enumerate(biclusters[data_class]):
            # Extract labels by indicator indexing.
            cluster_members[data_class][cluster_num] = {
                'cpgs': list(ref_data[data_class].index[bicluster[0]]),
                'genes': list(ref_data[data_class].columns[bicluster[1]])
            }
            
    return cluster_members

In [ ]:
def preds_to_disk(refs, clusters, model, parent='./../predictions/'):
    """Generate txt files containing row and column indicators for 
    detected biclusters associated with different datasets."""
    
    for ref_label in refs:

        stem = '{0}_biclusters_{1}.txt'.format(model, ref_label)
        with open(os.path.join(parent, stem), 'w') as outfile:   
            outfile.write('biclusters_{0}\n'.format(ref_label))

            for cluster_num, coords in clusters[ref_label].items():
                outfile.write('cluster_num_{0}\n'.format(cluster_num))
                outfile.write('{0}\n'.format(coords['cpgs']))
                outfile.write('{0}\n'.format(coords['genes']))
    
    return None

In [ ]:
def cluster_quality(biclusters):
    """Compute a metric to determine quality of biclusters."""
    
    #for cluster in biclusters:
    #    
    pass

In [ ]:
def cluster_similarity_score(true, pred):
    """Determines percentage of pred items in true."""
    
    return np.isin(pred, true).sum() / np.size(true)

In [ ]:
def cluster_stats(biclusters):
    """Compute max, min and std for a collection of 
    biclusters."""
    
    # Collect Statistical 
    #_cluster_stats = {}
    #for name, cluster in biclusters.items():
    #    _cluster_stats[num] = {
    #        'max': np.max(cluster), 
    #        'min': np.min(cluster),
    #        'std': np.std(cluster)
    #}
    #df_stats = pd.DataFrame(
    #    _cluster_stats, columns=['max', 'min', 'std']
    #)
    #df_stats.index = list(bicluster.keys())
    pass

## Clustering reference data

Applying the selected biclustering algorithms to the reference data, reconstructing and visualizing the results, selecting the bicluster members and writing the results to disk.

### Source: scikit-learn

In [ ]:
sk_models

In [ ]:
r_models

In [ ]:
# Reconstruct data matrices by sorting data according to
# predicted biclusters.
reconstr_data = {}
for ref_class in ref_labels:
    # Extract fitted model
    model = sk_clfs[ref_class]
    # Sort reference data
    _data = ref_data[ref_class].values
    _fit_data = _data[np.argsort(model.row_labels_)]
    _sorted_col_ids = np.argsort(model.column_labels_)
    reconstr_data[ref_class] = _fit_data[:, _sorted_col_ids]

### Mining Bonferroni corrected p-value

In [ ]:
gen_graphics(
    reconstr_data[ref_labels[0]],
    'Biclustering results of preprocessed\n'
    'Bonferroni corrected p-values', 
    './../predictions/imgs/org_prep_pvalues.png'
)

### Mining selected Bonferroni corrected p-values

Goal: Try to recreate the clusters and compare the contents to paper results.

In [ ]:
model = sk_clfs['sel_pvalues']
model.n_clusters = 2

reconstr_data = predict(
    sk_clfs['sel_pvalues'], 
    ref_data['sel_pvalues'].values
)

In [ ]:
gen_graphics(
    reconstr_data,
    'Biclustering results of selected preprocessed\n'
    'Bonferroni corrected p-values', 
    './../../predictions/imgs/sel_prep_pvalues.png'
)

In [ ]:
gen_graphics(
    reconstr_data[ref_labels[2]],
    'Biclustering results of preprocessed\n'
    'Pearson`s correlation coefficients', 
    './../predictions/imgs/org_prep_pcc.png'
)

In [ ]:
gen_graphics(
    reconstr_data[ref_labels[3]],
    'Biclustering results of selected preprocessed\n'
    'Pearson`s correlation coefficients', 
    './../predictions/imgs/sel_prep_pcc.png'
)

In [ ]:
# Fetch bicluster indicators for each detected bicluster
# stored as attributes in fitted models.
sk_biclusters_ = {}
for name, model in sk_clfs.items():
    sk_biclusters_[name] = cluster_indices(
        model.rows_, model.columns_
    )

In [ ]:
# Convert cluster indices to cpG and gene labels for each class 
# of reference data, and write results to disk.
sk_cluster_members = collect_cluster_members(
    sk_clfs, ref_labels, ref_data
)
preds_to_disk(ref_labels, sk_cluster_members, model='sk')

### Source: R

In [ ]:
# Collect pickled R models

r_clf_paths = [
    './../model_dumps/r_orig_prep_pvalues.pkl',
    './../model_dumps/r_sel_prep_pvalues.pkl',
    './../model_dumps/r_orig_prep_pcc.pkl',
    './../model_dumps/r_sel_prep_pcc.pkl',
]

r_clfs = collect_clfs(r_clf_paths, ref_labels)

In [ ]:
# Fit model to data producing cluster estiamtes.
for num, (_, model) in enumerate(r_clfs.items()):
    # NOTE: Ref data is <dict> (converting to <ndarray>)
    model.fit(list(ref_data.values())[num].values)

In [ ]:
# TODO: Visualizing results (use R tools)

In [ ]:
# Convert cluster indices to cpG and gene labels for each class 
# of reference data, and write results to disk.
r_cluster_members = collect_cluster_members(
    r_clfs, references, ref_data
)

preds_to_disk(references, r_cluster_members, model='r')

### Source: Binary

## Enrichment analysis

In [ ]:
# TODO: Checkout BiBench